## Aggregate H(x) IODA files on regular latitude/longitude
This notebook generate the gridded mean and standard deviation for obs, H(x), and obs - H(x) in NetCDF4 format. \
The number of pixels in a grid point is produced as well. \
Output variables are in the format: `{obsname}_{obs|hofx|omb}_{count|mean|std}`

In [ ]:
from tqdm import tqdm
import xarray as xa
import numpy as np
import pandas as pd
from functions import get_dates
from joblib import Parallel, delayed

### Setup the script running period, grid resolution, and input/output paths

In [ ]:
sdate = 2024110100
edate = 2024113018
date_interval = 6
dates = get_dates(sdate, edate, date_interval)
chnlidx = 4
varname = 'aerosolOpticalDepth'
lon_res = 0.625
lat_res = 0.5

ioda_path = '/glade/work/swei/Git/JEDI-METplus/output/aodobs_merra2/hofx/f00'
obs_name_list = [
    'modis_terra_aod', 'modis_aqua_aod', 'pace_aod',
    'viirs_aod_dt_npp', 'viirs_aod_dt_n20',
    'viirs_aod_db_npp', 'viirs_aod_db_n20',
]
output_gridded_file = '/glade/work/swei/projects/mmm.pace_aod/aod_obs_stats.m2.202411.nc4'

### Create the latitude and longitude for grid points and corresponding bounds

In [ ]:
latbin = np.arange(-90-0.5*lat_res, 90+lat_res, lat_res)
latgrd = np.arange(-90, 90+0.5*lat_res, lat_res)
lonbin = np.arange(-180-0.5*lon_res, 180, lon_res)
longrd = np.arange(-180, 180, lon_res)


def process_obsname(obsname):
    print(f'processing {obsname}')
    for i, cdate in tqdm(enumerate(dates)):
        cdate_str = cdate.strftime('%Y%m%d%H')
        iodafile = f'{ioda_path}/{obsname}/hofx.{obsname}.{cdate_str}.nc4'

        dims_ds = xa.open_dataset(iodafile)
        channel = dims_ds.Channel.values

        meta_ds = xa.open_dataset(iodafile, group='MetaData')
        lats = meta_ds.latitude.data
        lons = meta_ds.longitude.data

        obsv_ds = xa.open_dataset(iodafile, group='ObsValue').assign_coords(Channel=channel.astype(np.int32))
        obsv_data = obsv_ds[varname].sel(Channel=chnlidx).data
        hofx_ds = xa.open_dataset(iodafile, group='hofx').assign_coords(Channel=channel.astype(np.int32))
        hofx_data = hofx_ds[varname].sel(Channel=chnlidx).data
        ombs_data = obsv_data - hofx_data

        data_dict = {
            f'{obsname}_obs': (['locs'], obsv_data),
            f'{obsname}_hofx': (['locs'], hofx_data),
            f'{obsname}_omb': (['locs'], ombs_data),
            'lat': (['locs'], lats),
            'lon': (['locs'], lons),
        }

        coord_dict = {'locs': range(lons.size)}
        ds = xa.Dataset(data_dict, coords=coord_dict)

        df = ds.to_dataframe()
        df['latitude'] = pd.cut(df['lat'], bins=latbin, labels=latgrd)
        df['longitude'] = pd.cut(df['lon'], bins=lonbin, labels=longrd)
        grp = df.groupby(['latitude', 'longitude'], observed=False).agg({
            f'{obsname}_obs':['count', 'mean', 'std'],
            f'{obsname}_hofx':['mean', 'std'],
            f'{obsname}_omb':['mean', 'std'],
        })
        grp.columns = [f"{lvl1}_{lvl2}" for lvl1, lvl2 in grp.columns]
        gridded_ds = grp.to_xarray()

        if i == 0:
            outds = gridded_ds
        else:
            outds = xa.concat((outds, gridded_ds), dim='time')

    outds = outds.assign_coords({'time': dates})
    return outds

### Loop through observation in parallel

In [ ]:
dataset_list = Parallel(n_jobs=-1)(delayed(process_obsname)(obs) for obs in obs_name_list)
allds = xa.merge(dataset_list)
print('Process Finished')

### Write out the xarray dataset into a NetCDF file

In [ ]:
allds.to_netcdf(output_gridded_file)